# demo

## 获取学区地图数据

这里我们从网页获得学区的数据，对数据进行预处理，然后进入ArcMap中进行下一步处理

工具: Python,jupyter notebook,pycharm,notepad++,

以深圳市福田区为例，http://map.28dat.net/ftxx.aspx 提供了在线的学区的地图

首先打开网页，按F12，推荐Chrome浏览器

![网页](./page.jpg)

刷新后，

![网页](./pageRefresh.png)

ctrl F 查找数据在哪

![查找数据在哪](./search.png)

然后打开数据所在的网址，http://map.28dat.net/inc/ftxx.js 将数据复制粘贴下来,保存成文本

![数据截图](./dataSnapshot.png)

修改文本的扩展名为.json，然后在pycharm中，ctrl+Alt+L 可以让格式变得好看

并对不符合json格式的内容做修改，比如说key需要为字符串形式（需要引号），删除掉注释等

![pycharm批量修改](./pycharm.png)

如果在使用pandas存在乱码问题，可以使用notepad++把编码改为utf-8（注意window有可能会有utf-8 BOM，这个很坑的，改成utf-8）

![notepad++](./notepad.png)


In [1]:
import pandas as pd
# 读取json数据，转为dataframe
df = pd.read_json('futian.json',encoding='utf8')
df.head()

,name,no,point,polygon
0,荔园通新岭校区,1,"114.103493,22.552359","114.08804,22.55004;114.09334,22.550251;114.099..."
1,园岭小学,2,"114.104911,22.556381","114.102864,22.557443;114.10643,22.557577;114.1..."
2,福田小学,3,"114.089027,22.542053","114.083712,22.545811;114.083838,22.539836;114...."
3,新洲小学,4,"114.055607,22.530837","114.052447,22.533694;114.05258,22.53281;114.05..."
4,园岭实验,5,"114.108359,22.559802","114.1063,22.5715;114.10643,22.558628;114.11076..."


首先我们需要学校的坐标，要把坐标point分离成longitude和latitude

这里我使用了apply方法

In [45]:
# 对point列进行处理，apply方法对该列的每一个数都进行同样的处理
# apply方法需要的参数是一个方程，一个怎么样处理的方程，这里使用了 lambda 表达式。
# 相当于告诉，这个point是x，然后对point怎么处理
df['longitude'] = df['point'].apply(lambda x:x.split(',')[0])
df['latitude'] = df['point'].apply(lambda x:x.split(',')[-1])
df.head()

,name,no,point,polygon,longitude,latitude
0,荔园通新岭校区,1,"114.103493,22.552359","114.08804,22.55004;114.09334,22.550251;114.099...",114.103493,22.552359
1,园岭小学,2,"114.104911,22.556381","114.102864,22.557443;114.10643,22.557577;114.1...",114.104911,22.556381
2,福田小学,3,"114.089027,22.542053","114.083712,22.545811;114.083838,22.539836;114....",114.089027,22.542053
3,新洲小学,4,"114.055607,22.530837","114.052447,22.533694;114.05258,22.53281;114.05...",114.055607,22.530837
4,园岭实验,5,"114.108359,22.559802","114.1063,22.5715;114.10643,22.558628;114.11076...",114.108359,22.559802


In [46]:
# 保存成.csv格式，并且不要index
df.to_csv('futian_school.csv',index=0)

下面我要将polygon变成Arcmap能处理的形式，首先要把这一串东西变成point，点的格式如下表，然后Arcmap可以将这个处理成polyline然后再处理成polygon

school|lon|lat|lon1|lat1
--|:--|:--|:--|:
荔园通新岭校区|114.08804|22.55004|114.09334|22.550251
荔园通新岭校区|114.09334|22.550251|114.0997|22.550451
荔园通新岭校区|114.0997|22.550451|114.09965|22.55492

In [38]:
list = [] #用一个列表保存处理过后的信息，后面用这个list创建dataFrame
for index, row in df.iterrows():
#     遍历刚才df的每一行，每一行是一个学校
    points = row['polygon'].split(';') #这里取出了polygon变量，并且把字符串用；分开，变成由字符串组成的列表
    for i in range(0,len(points)):
        school_name = row['name'] #取出学校那个变量
        pos = points[i].split(',') #把lon和lat分开
        lon = pos[0]
        lat = pos[-1]
        if i < len(points)-1: #对于非最后一次循环来说，因为最后一次循环是没有 i+1 的
            pos1 = points[i+1].split(',')
            lon1 = pos1[0]
            lat1 = pos1[-1]
        else: #对于最后一次循环
            pos1 = points[0].split(',')
            lon1 = pos1[0]
            lat1 = pos1[-1]
        dic = {'school':school_name,'lon':lon,'lat':lat,'lon1':lon1,'lat1':lat1} #将这次循环的结果保存在字典中
        list.append(dic) #将字典添加到列表中去

[{'school': '荔园通新岭校区', 'lon': '114.08804', 'lat': '22.55004', 'lon1': '114.09334', 'lat1': '22.550251'}, {'school': '荔园通新岭校区', 'lon': '114.09334', 'lat': '22.550251', 'lon1': '114.0997', 'lat1': '22.550451'}, {'school': '荔园通新岭校区', 'lon': '114.0997', 'lat': '22.550451', 'lon1': '114.09965', 'lat1': '22.55492'}, {'school': '荔园通新岭校区', 'lon': '114.09965', 'lat': '22.55492', 'lon1': '114.11075', 'lat1': '22.5552'}, {'school': '荔园通新岭校区', 'lon': '114.11075', 'lat': '22.5552', 'lon1': '114.110877', 'lat1': '22.547253'}]


In [42]:
df1 = pd.DataFrame(list,columns=['school','lon','lat','lon1','lat1'])
df1.head(11)

,school,lon,lat,lon1,lat1
0,荔园通新岭校区,114.08804,22.55004,114.09334,22.550251
1,荔园通新岭校区,114.09334,22.550251,114.0997,22.550451
2,荔园通新岭校区,114.0997,22.550451,114.09965,22.55492
3,荔园通新岭校区,114.09965,22.55492,114.11075,22.5552
4,荔园通新岭校区,114.11075,22.5552,114.110877,22.547253
5,荔园通新岭校区,114.110877,22.547253,114.10266,22.54699
6,荔园通新岭校区,114.10266,22.54699,114.10259,22.5506
7,荔园通新岭校区,114.10259,22.5506,114.0996,22.5505
8,荔园通新岭校区,114.0996,22.5505,114.099684,22.54688
9,荔园通新岭校区,114.099684,22.54688,114.08808,22.546395


In [43]:
df1.to_csv('futian_line.csv',index=0)

# Arcmap处理

## 学区图的构建

**Make XY Event Layer**
：将表格（.csv,.xls）转换为事件图层

**Copy Features**
：将事件图层转换为 .shp 文件

**XY To Line**
：将我们上面做的lon,lat,lon1,lat1转换为线，输入是表格

**Feature To Polygon**
：讲线转换为多边形

**Feature To Point**
：将线，多边形等转换为点

这里我们需要Feature To Point的原因是，我们需要这个点的图层为polygon做标签，不然polygon就不会有标签（不知道哪个polygon对应哪个学校）

![model](./model.png)

把.shp文件拖Arcmap的窗口就可以见到结果了

![schoolMap](./schoolMap.png)

更多关于arcmap在空间经济学中应用，可以看这里的tutorial https://github.com/mkudamatsu/mkudamatsu.github.io

## 将住房数据投影到地图上

我这里提供了深圳的部分数据作为演示

![深圳住房数据截图](./housingDataSnapshot.png)

结果

![住房地图](./school_housing.png)

接下来我们要判断住房在哪个学区中，Arcmap里面有这样的工具

**spatial join**
：将要素和其他要素拼起来

这里我们就需要把住房数据（点）和学区图（多边形）拼到一起去，这样我们就可以判断出点在哪个多边形上了

**table to excel** 
： 将.shp转换为xls表格保存

模型
![模型2](./model2.png)



## 用高德地图api和百度地图api获取地点的坐标

### 从地址到坐标

首先建议大家去自己获得一个高德地图api和百度地图api的key。去官网就可以获得轻松获得。下面的key是我个人的，仅做演示用。

### api都有些什么功能

一般我们能用上的就只有，地址 -> 坐标 和 坐标-> 坐标

这里我以获得福田区的地铁站座标为例子。我首先用高德地图的api获得坐标（gjc02），然后用百度坐标的api转换为百度坐标（bd09）（因为我们学区的底图用的是bd09，我们需要保持一致）

In [80]:
# https://lbs.amap.com/api/webservice/guide/api/georegeo
# http://lbsyun.baidu.com/index.php?title=webapi/guide/changeposition
# 福邻地铁站 暂未开通 会查询不到坐标
# 下面的key是我个人的key，大家使用的时候，自己去官网上申请一个key吧
baidu_key = 'nCm54poohZ4dzDCuqKa3q9nfDkC6R4IZ'
gaode_key = '0d108d0b54a6d04fa716eb45632b04b0'
subway_list = ['华强路站','岗厦站','会展中心站','购物公园站','香蜜湖站','车公庙站','竹子林站','侨城东站','益田站','石厦站','少年宫站',
               '莲花村站','华新站','通新岭站','红岭站','福田口岸站','福民站','市民中心站','莲花北站','上梅林站','安托山站','农林站',
               '上沙站','沙尾站','皇岗村站','皇岗口岸站','赤尾站','华强南站','华强北站','华新站','黄木岗站',
               '八卦岭站','红岭北站','景田站','莲花西站','岗厦北站','华强北站','燕南站']

In [72]:
import requests
params = {'address': '上沙地铁站',
          'output':'JSON',
          'city': '深圳市',
          'key' : gaode_key}

req  = requests.get('https://restapi.amap.com/v3/geocode/geo',params = params)
print(req.url)
dic = req.json()
print('\n高德地图返回:\n',dic)
print('\n坐标提取出来：\n',dic['geocodes'][0]['location'])
params1 = {'coords':dic['geocodes'][0]['location'],
           'from':3,
           'to':5,
          'output':'json',
          'ak' : baidu_key}
req1 = requests.get('http://api.map.baidu.com/geoconv/v1/',params1)
dic1 = req1.json()
print(dic1)
dic1['result'][0]['x'],dic1['result'][0]['y']

https://restapi.amap.com/v3/geocode/geo?address=%E4%B8%8A%E6%B2%99%E5%9C%B0%E9%93%81%E7%AB%99&output=JSON&city=%E6%B7%B1%E5%9C%B3%E5%B8%82&key=0d108d0b54a6d04fa716eb45632b04b0
高德地图返回:
 {'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区上沙地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '440304', 'street': [], 'number': [], 'location': '114.035504,22.524612', 'level': '兴趣点'}]}
坐标提取出来：
 114.035504,22.524612
{'status': 0, 'result': [{'x': 114.04200436025242, 'y': 22.53051104550219}]}


(114.04200436025242, 22.53051104550219)

In [75]:
def address_to_baidu_corrds(address,gaode_key,baidu_key):
    """
    把地址转换为百度坐标。
    先使用高德地图api转换将地址转换为gjc02坐标，然后再用百度地图api转换为bd09坐标
    """
    params = {'address':address,
          'output':'JSON',
          'city': '深圳市',
          'key' : gaode_key}
    req  = requests.get('https://restapi.amap.com/v3/geocode/geo',params = params)
    dic = req.json()
    params1 = {'coords':dic['geocodes'][0]['location'],
               'from':3,
               'to':5,
              'output':'json',
              'ak' : baidu_key}
    req1 = requests.get('http://api.map.baidu.com/geoconv/v1/',params1)
    dic1 = req1.json()
    return dic1['result'][0]['x'],dic1['result'][0]['y']

In [81]:
result_list = []
for subway in subway_list:
    s = subway.replace('站','地铁站')
    t = address_to_baidu_corrds(s,gaode_key,baidu_key)
    d = {'subway':s, 'lon':t[0],'lat':t[1]}
    print(d)
    result_list.append(d)

{'address': '华强路地铁站', 'output': 'JSON', 'city': '深圳市', 'key': '0d108d0b54a6d04fa716eb45632b04b0'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区华强路地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '440304', 'street': [], 'number': [], 'location': '114.085133,22.540459', 'level': '兴趣点'}]}
{'subway': '华强路地铁站', 'lon': 114.09163329502327, 'lat': 22.54636941239971}
{'address': '岗厦地铁站', 'output': 'JSON', 'city': '深圳市', 'key': '0d108d0b54a6d04fa716eb45632b04b0'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区岗厦地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '440304', 

{'address': '福田口岸地铁站', 'output': 'JSON', 'city': '深圳市', 'key': '0d108d0b54a6d04fa716eb45632b04b0'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区福田口岸地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '440304', 'street': [], 'number': [], 'location': '114.069273,22.515737', 'level': '兴趣点'}]}
{'subway': '福田口岸地铁站', 'lon': 114.07581834074897, 'lat': 22.521431500102754}
{'address': '福民地铁站', 'output': 'JSON', 'city': '深圳市', 'key': '0d108d0b54a6d04fa716eb45632b04b0'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区福民地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '44030

{'subway': '华新地铁站', 'lon': 114.09315508852293, 'lat': 22.55491922827678}
{'address': '黄木岗地铁站', 'output': 'JSON', 'city': '深圳市', 'key': '0d108d0b54a6d04fa716eb45632b04b0'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区黄木岗地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '440304', 'street': [], 'number': [], 'location': '114.087280,22.555843', 'level': '兴趣点'}]}
{'subway': '黄木岗地铁站', 'lon': 114.0937853676918, 'lat': 22.56179358234444}
{'address': '八卦岭地铁站', 'output': 'JSON', 'city': '深圳市', 'key': '0d108d0b54a6d04fa716eb45632b04b0'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '广东省深圳市福田区八卦岭地铁站', 'country': '中国', 'province': '广东省', 'citycode': '0755', 'city': '深圳市', 'district': '福田区', 'township': [], 'neighborhood': {'name': [

In [88]:
subway_pd = pd.DataFrame(result_list,columns=['subway','lon','lat'])
subway_pd.to_csv('sz_subway_cor.csv',index=0)
subway_pd.head()

,subway,lon,lat
0,华强路地铁站,114.091633,22.546369
1,岗厦地铁站,114.073382,22.540787
2,会展中心地铁站,114.068907,22.540416
3,购物公园地铁站,114.061016,22.540047
4,香蜜湖地铁站,114.045149,22.544734


## 计算房子到最近的地铁站的距离

在hedonic pricing model中，很多文献会以到达最近的地铁站的距离作为一个这个地方方便程度的proxy variable（代理变量）

我们在这节中讲如何计算

大致的思路就是，有房子的坐标，有地铁的坐标，然后使用公式计算出到达每个地铁站的距离，然后取最小的那个

首先我们先定义一下计算距离的公式，这个公式有别于**欧氏距离**的公式，因为地球是个球体（不过距离稍微近一点，影响不大）



<center> <font size=4> $𝑑_{𝑖𝑗} = 111.12×𝑐𝑜𝑠^{−1} [sin(𝑙𝑎𝑡_𝑖)sin(𝑙𝑎𝑡_𝑗)+ cos(𝑙𝑎𝑡_𝑖)cos(𝑙𝑎𝑡_𝑗)cos(𝑙𝑜𝑛_𝑖−𝑙𝑜𝑛_𝑗)]$ </font></center>



In [2]:
from math import *
def great_circle_distance(coord_i,coord_j):
    return 111.12 * acos( sin(coord_i[1])*sin(coord_j[1])
                         + cos(coord_i[1]) * cos(coord_j[1]) * cos(coord_i[0] - coord_j[0]))    

In [7]:
housing_school_df = pd.read_excel('sz_housing_school.xls',na_values=' ')
housing_school_df.dropna(inplace=True,subset=['name'])
housing_school_df.drop(columns = ['FID','Join_Count','TARGET_FID','no','point','polygon'],inplace=True,axis = 1)
housing_school_df.sort_values(by = '小区',inplace = True)

subway_df = pd.read_csv('sz_subway_cor.csv')

housing_school_df

,name,longitude,latitude,lianjia_id,总价,单价,户型,楼层,朝向,装修,...,lat,url,成交时间,挂牌价格,成交周期,调价次数,带看次数,关注,浏览,小区域
9816,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101566000,252万,89872,1室0厅1厨1卫,中楼层(共29层),东南,其他,...,22.558105,https://sz.lianjia.com/chengjiao/105101565676....,2018.09.23,255.0,3,1,0,0,11,景田
2502,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101120000,335万,74528,1室1厅1厨1卫,中楼层(共29层),北,精装,...,22.558105,https://sz.lianjia.com/chengjiao/105101119800....,2018.04.14,348.0,136,0,0,13,2056,景田
8833,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101444000,248万,87819,1室0厅1厨1卫,高楼层(共30层),北,精装,...,22.558105,https://sz.lianjia.com/chengjiao/105101444371....,2018.07.29,250.0,18,1,0,0,116,景田
3901,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101202000,430万,81133,2室1厅1厨1卫,中楼层(共30层),南,其他,...,22.558105,https://sz.lianjia.com/chengjiao/105101202190....,2018.01.24,450.0,2,0,0,0,0,景田
1439,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101014000,415万,79244,2室1厅1厨1卫,中楼层(共29层),东,简装,...,22.558105,https://sz.lianjia.com/chengjiao/105101014476....,2018.01.13,430.0,114,1,0,9,2012,景田
4999,景田小学,114.041996,22.562822,105101251000,293万,67264,1室1厅1厨1卫,中楼层(共33层),南,其他,...,22.563578,https://sz.lianjia.com/chengjiao/105101250531....,2018.04.07,300.0,31,0,0,2,656,香梅北
8753,景田小学,114.041996,22.562822,105101438000,220万,73334,1室1厅1厨1卫,高楼层(共33层),北,其他,...,22.563578,https://sz.lianjia.com/chengjiao/105101437982....,2018.10.20,225.0,105,1,0,9,1819,香梅北
5997,景田小学,114.041996,22.562822,105101285000,216万,72000,1室0厅1厨1卫,中楼层(共33层),北,其他,...,22.563578,https://sz.lianjia.com/chengjiao/105101285351....,2018.03.30,230.0,2,0,0,0,0,香梅北
8913,上步小学,114.105272,22.543964,105101450000,702万,57679,4室1厅1厨2卫,高楼层(共32层),东南,其他,...,22.545818,https://sz.lianjia.com/chengjiao/105101450073....,2018.07.21,790.0,6,0,0,0,150,上步
8020,福华小学,114.076699,22.537030,105101392000,495万,50552,3室1厅1厨2卫,高楼层(共23层),西南,简装,...,22.536672,https://sz.lianjia.com/chengjiao/105101391836....,2018.07.23,499.0,49,1,0,48,3352,福田中心


In [8]:
# 方法一：遍历两个DataFrame
housing_school_df['distance_to_subway'] = float('inf')
last_community = ''
last_dist = float('inf')
for index, row in housing_school_df.iterrows():
    coord_i = (row['lon'],row['lat'])
    # 这里为了减少计算量，因为之前给df根据小区进行的排序，如果下一行的小区相同，就直接赋值距离，而不重新计算    
    if row['小区']==last_community:
        housing_school_df['distance_to_subway'][index] = last_dist
        continue #跳过下面内容，进入下一次循环
    min_dist = float('inf')
    ss = ''
    # 遍历地铁站的df，求到每个地铁站的距离，取最小     
    for index1, row1 in subway_df.iterrows():
        coord_j = (row1['lon'],row['lat'])
        dis = great_circle_distance(coord_i,coord_j)
        min_dist = min(min_dist,dis)
        if min_dist==dis:
            ss = row1['subway']
    # 记录该小区和该小区到地铁站的最小距离    
    last_community = row['小区']
    last_dist = min_dist
    print('离 {} 最近的地铁站是：{} 距离为：{}km'.format(row['小区'],ss,min_dist))
    # 把值填上去    
    housing_school_df['distance_to_subway'][index] = min_dist    

离 TT国际公寓 最近的地铁站是：沙尾地铁站 距离为：0.14078517803066523km
离 七街公馆 最近的地铁站是：上沙地铁站 距离为：0.2944495863584193km


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


离 万德居 最近的地铁站是：红岭地铁站 距离为：0.013659616422794965km
离 万景花园 最近的地铁站是：福田口岸地铁站 距离为：0.0707363574887975km
离 万科温馨家园 最近的地铁站是：农林地铁站 距离为：0.13683272251098885km
离 万科金域蓝湾一期 最近的地铁站是：车公庙地铁站 距离为：0.007227565287369277km
离 万科金域蓝湾三期 最近的地铁站是：车公庙地铁站 距离为：0.14743104418778283km
离 万科金域蓝湾二期 最近的地铁站是：车公庙地铁站 距离为：0.13577238258464067km
离 万科金色家园 最近的地铁站是：景田地铁站 距离为：0.07469614580436931km
离 上步码头小区 最近的地铁站是：华强路地铁站 距离为：0.006339039842042184km
离 世纪豪庭 最近的地铁站是：农林地铁站 距离为：0.307330311972637km
离 东华大厦 最近的地铁站是：福田口岸地铁站 距离为：0.1300528265629374km
离 东怡大厦 最近的地铁站是：福田口岸地铁站 距离为：0.1082589498719687km
离 东方园博苑 最近的地铁站是：侨城东地铁站 距离为：0.39888047661721393km
离 东方时代广场 最近的地铁站是：华新地铁站 距离为：0.04113612856468703km
离 东方欣悦居 最近的地铁站是：购物公园地铁站 距离为：0.029552842444693216km
离 东景花园 最近的地铁站是：莲花西地铁站 距离为：0.15744454172995706km
离 东海国际公寓 最近的地铁站是：农林地铁站 距离为：0.33145036104692327km
离 东海花园 最近的地铁站是：车公庙地铁站 距离为：0.3044875875185123km
离 东海花园二期福禄居 最近的地铁站是：车公庙地铁站 距离为：0.04804154359040288km
离 中泰燕南名庭 最近的地铁站是：燕南地铁站 距离为：0.0724251548467822km
离 中海华庭 最近的地铁站是：益田地铁站 距离为：0.08855269130187518km
离 中港城 最近的地铁站是：石厦地

离 景发花园 最近的地铁站是：香蜜湖地铁站 距离为：0.05432121721345286km
离 景尚雅苑 最近的地铁站是：上沙地铁站 距离为：0.14872692679421223km
离 景新花园 最近的地铁站是：莲花西地铁站 距离为：0.23616305035928475km
离 景洲大厦 最近的地铁站是：莲花西地铁站 距离为：0.17813397806218018km
离 景源华庭 最近的地铁站是：莲花村地铁站 距离为：0.011731303936073302km
离 景田北村 最近的地铁站是：景田地铁站 距离为：0.07259731349940676km
离 景秀年华家园 最近的地铁站是：上沙地铁站 距离为：0.4145479035972697km
离 景蜜村 最近的地铁站是：香蜜湖地铁站 距离为：0.004304845651031192km
离 景鹏大厦 最近的地铁站是：景田地铁站 距离为：0.24343757345215217km
离 朗庭豪园 最近的地铁站是：皇岗村地铁站 距离为：0.011710759378611739km
离 朗晴馨洲 最近的地铁站是：景田地铁站 距离为：0.20006611009487762km
离 朝恒大厦 最近的地铁站是：福民地铁站 距离为：0.0402803294152745km
离 杭钢富春大厦 最近的地铁站是：农林地铁站 距离为：0.24642511829391545km
离 杰恩大厦 最近的地铁站是：八卦岭地铁站 距离为：0.019617080833617923km
离 林海山庄 最近的地铁站是：上沙地铁站 距离为：0.3065048413379502km
离 核电花园 最近的地铁站是：燕南地铁站 距离为：0.2365071077223408km
离 桐林公寓 最近的地铁站是：皇岗口岸地铁站 距离为：0.4308882630223561km
离 桑达大厦 最近的地铁站是：黄木岗地铁站 距离为：0.21325153915501172km
离 桑达小区 最近的地铁站是：燕南地铁站 距离为：0.18701512077493596km
离 桑达雅苑 最近的地铁站是：黄木岗地铁站 距离为：0.21956437203786855km
离 梅兴苑 最近的地铁站是：莲花西地铁站 距离为：0.03241875790476279km


离 金海丽名居 最近的地铁站是：沙尾地铁站 距离为：0.04424913893659218km
离 金海湾花园 最近的地铁站是：上沙地铁站 距离为：0.33187325864846645km
离 金海燕花园 最近的地铁站是：侨城东地铁站 距离为：0.022898428415863575km
离 金海花园 最近的地铁站是：安托山地铁站 距离为：0.0880912521530313km
离 金港豪庭 最近的地铁站是：福民地铁站 距离为：0.03523274147805781km
离 金燕园 最近的地铁站是：市民中心地铁站 距离为：0.006298170802987167km
离 金色假日 最近的地铁站是：景田地铁站 距离为：0.11058291075245376km
离 金茂礼都 最近的地铁站是：燕南地铁站 距离为：0.11496986392073535km
离 银海苑 最近的地铁站是：福民地铁站 距离为：0.02017222899178706km
离 锦州花园 最近的地铁站是：景田地铁站 距离为：0.11891537508558338km
离 锦庐花园 最近的地铁站是：安托山地铁站 距离为：0.17612084201697875km
离 锦林新居 最近的地铁站是：香蜜湖地铁站 距离为：0.09776425490329885km
离 锦滨苑 最近的地铁站是：沙尾地铁站 距离为：0.028580687436494726km
离 长乐花园 最近的地铁站是：燕南地铁站 距离为：0.14428381470905152km
离 长城二花园 最近的地铁站是：黄木岗地铁站 距离为：0.06809825920310195km
离 长城盛世家园一期 最近的地铁站是：岗厦地铁站 距离为：0.019807617751439955km
离 长城盛世家园二期 最近的地铁站是：福民地铁站 距离为：0.005885507948256288km
离 长城花园 最近的地铁站是：黄木岗地铁站 距离为：0.24395831306580595km
离 长安花园 最近的地铁站是：燕南地铁站 距离为：0.2269178536282959km
离 长怡花园 最近的地铁站是：黄木岗地铁站 距离为：0.09958534159271291km
离 长泰花园 最近的地铁站是：黄木岗地铁站 距离为：0.01065307409

In [15]:
# 方法二：用apply和lambda 表达式，快很多
def nearest_dis(x,another_df):
    dis = float('inf')
    another_df['temp'] = another_df.apply(lambda y:great_circle_distance((x['lon'],x['lat']),(y['lon'],y['lat'])),axis=1)
    return another_df['temp'].min()

housing_school_df['distance_to_subway'] = housing_school_df.apply(lambda x:nearest_dis(x,subway_df),axis = 1)

In [16]:
housing_school_df['distance_to_school'] = housing_school_df.apply(
    lambda x: great_circle_distance((x['lon'],x['lat']),(x['longitude'],x['latitude'])),axis = 1)
housing_school_df.to_csv('housing_school_dis.csv',index=0)
housing_school_df

,name,longitude,latitude,lianjia_id,总价,单价,户型,楼层,朝向,装修,...,成交时间,挂牌价格,成交周期,调价次数,带看次数,关注,浏览,小区域,distance_to_subway,distance_to_school
9816,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101566000,252万,89872,1室0厅1厨1卫,中楼层(共29层),东南,其他,...,2018.09.23,255.0,3,1,0,0,11,景田,0.245426,0.350473
2502,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101120000,335万,74528,1室1厅1厨1卫,中楼层(共29层),北,精装,...,2018.04.14,348.0,136,0,0,13,2056,景田,0.245426,0.350473
8833,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101444000,248万,87819,1室0厅1厨1卫,高楼层(共30层),北,精装,...,2018.07.29,250.0,18,1,0,0,116,景田,0.245426,0.350473
3901,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101202000,430万,81133,2室1厅1厨1卫,中楼层(共30层),南,其他,...,2018.01.24,450.0,2,0,0,0,0,景田,0.245426,0.350473
1439,荔外狮岭校区/荔外天骄校区,114.048338,22.555013,105101014000,415万,79244,2室1厅1厨1卫,中楼层(共29层),东,简装,...,2018.01.13,430.0,114,1,0,9,2012,景田,0.245426,0.350473
4999,景田小学,114.041996,22.562822,105101251000,293万,67264,1室1厅1厨1卫,中楼层(共33层),南,其他,...,2018.04.07,300.0,31,0,0,2,656,香梅北,1.140157,0.305517
8753,景田小学,114.041996,22.562822,105101438000,220万,73334,1室1厅1厨1卫,高楼层(共33层),北,其他,...,2018.10.20,225.0,105,1,0,9,1819,香梅北,1.140157,0.305517
5997,景田小学,114.041996,22.562822,105101285000,216万,72000,1室0厅1厨1卫,中楼层(共33层),北,其他,...,2018.03.30,230.0,2,0,0,0,0,香梅北,1.140157,0.305517
8913,上步小学,114.105272,22.543964,105101450000,702万,57679,4室1厅1厨2卫,高楼层(共32层),东南,其他,...,2018.07.21,790.0,6,0,0,0,150,上步,1.068289,0.473775
8020,福华小学,114.076699,22.537030,105101392000,495万,50552,3室1厅1厨2卫,高楼层(共23层),西南,简装,...,2018.07.23,499.0,49,1,0,48,3352,福田中心,0.547828,0.041826


到这里，本次教程就完了。

后面的内容就是去Stata等计量分析的软件进行计量建模等等。这里提供了最基本的，可能会用上的一些工具和函数等等。

里面的代码几乎都能再重复使用，代码也可以随便复制、随便传播。

希望能帮助到大家。